In [3]:
import random
import networkx as nx
import numpy as np
import karateclub
from tkinter import _flatten
import community
import MGCF

In [5]:
import pandas as pd

### 1~4. Data Generation

In [1]:
sample_num = 2000

In [18]:
# 构造图节点
nodes_list = list(range(sample_num))
G1 = nx.Graph()
G1.add_nodes_from(nodes_list)

# 读取邻接矩阵adjacent
# 以工作日(周一)的早高峰时段为例
adj_modul = pd.read_csv('../../data/adj/adj_20151109_0002.csv')
adj_modul = adj_modul.iloc[:, 1:].values
adj_modul = adj_modul[:sample_num:, :sample_num:]
adj_modul = adj_modul + adj_modul.T
adj_modul = adj_modul - np.diag(adj_modul.diagonal())
# for i in range(sample_num):
#     if (adj_modul[i]==0).all():
#         adj_modul[i, :] = 0.0001
#         adj_modul[:, i] = 0.0001
    
D_12 = np.diag(np.power(np.sum(adj_modul, axis=0)+0.00001, -1/2))
adj_modul_norm = np.dot(np.dot(D_12, adj_modul), D_12)

for i in range(sample_num):
    for j in range(sample_num):
        tmp = adj_modul_norm[i, j]
        if tmp>0: G1.add_edge(i, j, weight=tmp)

In [19]:
adj_reach = pd.read_csv('../../data/adj_reachability/adj_reachability.csv')
adj_reach = adj_reach.iloc[::, 1::].values
adj_reach = adj_reach[:sample_num:, :sample_num:]
adj_reach = adj_reach - np.diag(adj_reach.diagonal())
# for i in range(sample_num):
#     if (adj_reach[i]==0).all():
#         adj_reach[i, :] = 0.0001
#         adj_reach[:, i] = 0.0001
    
G2 = nx.Graph()
G2.add_nodes_from(nodes_list)

D_12 = np.diag(np.power(np.sum(adj_reach, axis=0)+0.00001, -1/2))
adj_reach_norm = np.dot(np.dot(D_12, adj_reach), D_12)

for i in range(sample_num):
    for j in range(sample_num):
        tmp = adj_reach_norm[i, j]
        if tmp>0: G2.add_edge(i, j, weight=tmp)

In [8]:
feature = pd.read_csv('../../data/features/features_wf_new.csv')
feature = feature.iloc[:, 1::].values
feature = feature[:sample_num:, :]

### 5. Combine Multi-Graph

In [20]:
G = nx.Graph()
G.add_nodes_from(nodes_list)

adj_combine = adj_modul_norm + adj_reach_norm
for i in range(sample_num):
    if (adj_combine[i]==0).all():
        adj_combine[i, :] = 0.0001
        adj_combine[:, i] = 0.0001

D_12 = np.diag(np.power(np.sum(adj_combine, axis=0)+0.00001, -1/2))
adj_combine_norm = np.dot(np.dot(D_12, adj_combine), D_12)

for i in range(sample_num):
    for j in range(sample_num):
        tmp = adj_combine_norm[i, j]
        if tmp>0: G.add_edge(i, j, weight=tmp)

### 6. for Algorithm Process & Evaluation

In [29]:
N = sample_num
X = feature
M = feature.shape[1]
K = 17
# omega = 1/K*np.ones(K)
beta = 1/M*np.ones(M)

In [59]:
# 1. DANMF

model1 = karateclub.DANMF()
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.582
0.561


In [58]:
# 2. NNSED

model1 = karateclub.NNSED(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

-0.0
0.706


In [57]:
# 3. MNMF

model1 = karateclub.MNMF(clusters=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.357
0.04


In [56]:
# 4. BigClam

model1 = karateclub.BigClam(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.005
0.67


In [55]:
# 5. SNMF

model1 = karateclub.SymmNMF(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

-0.006
0.042


In [54]:
# 6. GEMSEC

model1 = karateclub.GEMSEC(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.292
0.63


In [53]:
# 7. SCD

model1 = karateclub.SCD()
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.425
0.183


In [52]:
# 8. LPA

model1 = karateclub.LabelPropagation(seed=10, iterations=100)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

-0.0
0.0


In [48]:
# 9. Louvain

partition = community.best_partition(G, weight='weight', randomize=True, resolution=2)
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.783
0.13


In [50]:
# Ours
partition = MGCF.main(clusters_number=K, feature=X, multi_graph=[G1, G2], alpha_1=1, alpha_2=0.1, alpha_3=0.001, weight_list=[1, 1], alpha=2, epoch_num=700)

Epoch:  0 ; soft_mod:  -0.002904 ; loss_cons:  0.057599 ; loss_clus:  0.0
Epoch:  10 ; soft_mod:  -0.000283 ; loss_cons:  0.037043 ; loss_clus:  1.84574
Epoch:  20 ; soft_mod:  0.002731 ; loss_cons:  0.029711 ; loss_clus:  1.815098
Epoch:  30 ; soft_mod:  0.002473 ; loss_cons:  0.034163 ; loss_clus:  1.515752
Epoch:  40 ; soft_mod:  0.009534 ; loss_cons:  0.029763 ; loss_clus:  1.28132
Epoch:  50 ; soft_mod:  0.006305 ; loss_cons:  0.021884 ; loss_clus:  1.105752
Epoch:  60 ; soft_mod:  0.024029 ; loss_cons:  0.023747 ; loss_clus:  0.979124
Epoch:  70 ; soft_mod:  0.037242 ; loss_cons:  0.026684 ; loss_clus:  0.871779
Epoch:  80 ; soft_mod:  0.055106 ; loss_cons:  0.02381 ; loss_clus:  0.814244
Epoch:  90 ; soft_mod:  0.084603 ; loss_cons:  0.023199 ; loss_clus:  0.774688
Epoch:  100 ; soft_mod:  0.155083 ; loss_cons:  0.021722 ; loss_clus:  0.774612
Epoch:  110 ; soft_mod:  0.243362 ; loss_cons:  0.025372 ; loss_clus:  0.796897
Epoch:  120 ; soft_mod:  0.307496 ; loss_cons:  0.024007 

In [51]:
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round((community.modularity(partition, G1, weight='weight') + community.modularity(partition, G2, weight='weight'))/2, 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.767
0.123


In [40]:
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.813
0.114
